In [1]:
pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import urllib.request 
from mip import Model, xsum, maximize, BINARY, LinExpr

In [3]:
#criando matriz de adjacência das relações entre os usuários

graph = urllib.request.urlopen('https://raw.githubusercontent.com/natalialidia/CTFP-model/main/test-instances/S1.dat').read().decode('utf-8').split("\r\n")
del graph[-1] #vazio

numUsers = int(graph[0].split(" ")[0])
relationships = np.zeros(numUsers*numUsers, dtype=np.int64).reshape(numUsers,numUsers)

for line in graph[1:]:
  l = [int(x) for x in line.split(" ") if x]
  if l[0] < numUsers and l[1] < numUsers:
    relationships[l[0],l[1]] = relationships[l[1],l[0]] = int(l[2])

In [4]:
#definindo conjunto de skills de cada usuário

skillsData = urllib.request.urlopen('https://raw.githubusercontent.com/natalialidia/CTFP-model/main/test-instances/K1.txt').read().decode('utf-8').split("\n")
del skillsData[-1] #vazio

numSkills = int(skillsData[0].split(" ")[0])

def userSkillSet(user):
  return np.asarray([id for id,s in enumerate(skillsData[user+1].split(" ")) if s and int(s) == 1])

In [5]:
#definindo demanda de usuários por skills para cada time

teamsData = urllib.request.urlopen('https://raw.githubusercontent.com/natalialidia/CTFP-model/main/test-instances/R5.txt').read().decode('utf-8').split("\n")
del teamsData[-1] #vazio

numTeams = int(teamsData[0].split(" ")[0])

teamDemand = np.zeros(numTeams*numSkills, dtype=np.int64).reshape(numTeams, numSkills)

for id, line in enumerate(teamsData[1:]):
  l = [int(x) for x in line.split("\t") if x]
  teamDemand[id] = np.asarray(l)

print(teamDemand)

#conjunto de skills que cada time precisa
def demandSkillSet(team):
  return np.asarray([id for id,s in enumerate(teamDemand[team]) if s > 0])

[[10 10 10 10 10]
 [10 10 10 10 10]]


In [6]:
model = Model("CTFP")

#variáveis
x = [[[model.add_var(var_type=BINARY) for s in range(numSkills)] for u in range(numUsers)] for t in range(numTeams)]

#função objetivo
model.objective = maximize(xsum( xsum( xsum( x[t][u][s] for s in userSkillSet(u) ) for u in range(numUsers) ) for t in range(numTeams)))

#restrições

#restrição (63)
for t in range(numTeams):
  for u in range(numUsers):
    for v in range(numUsers):
      if relationships[u,v] == -1:
        model += xsum(x[t][u][s] for s in userSkillSet(u)) + xsum(x[t][v][s] for s in userSkillSet(v)) <= 1

#restrição (64)
for t1 in range(numTeams):
  for t2 in np.arange(t1+1, numTeams, 1):
    for u in range(numUsers):
      for v in range(numUsers):
        if relationships[u,v] == 1:
          model += xsum(x[t1][u][s] for s in userSkillSet(u)) + xsum(x[t2][v][s] for s in userSkillSet(v)) <= 1

#restrição (65)
for u in range(numUsers):
  model += xsum( xsum( x[t][u][s] for s in userSkillSet(u)) for t in range(numTeams)) <= 1

#restrição (66)
for t in range(numTeams):
  for s in demandSkillSet(t):
    model += xsum( x[t][u][s] for u in range(numUsers) if s in userSkillSet(u)) >= teamDemand[t,s]

model.optimize()

print(f'Valor objetivo: {model.objective_value}')

Valor objetivo: 219.0
